# Trouver et appliquer les règles à entropie nulle


## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs,glob
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import networkx as nx
import pickle,yaml
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

### Préparation des matrices de traits

In [2]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [122]:
import unicodedata
def listerUnicode(chaine):
    result=[]
    comb=False
    prec=u""
    for char in chaine:
        print char,
        if unicodedata.combining(char):
            result.append(prec+char)
            prec=""
        else:
            result.append(prec)
            prec=char
    result.append(prec)
    return [r for r in result if r!=""]

def coderIPA(chaine,table=toipa):
    result=chaine
    for k in table:
        result=result.replace(k,table[k])
    return result

In [123]:
ipaIn = listerUnicode(u"SZNêôâ")
ipaOut= [u"ʃ",u"ʒ",u"ŋ",u"ɛ̃̃ ",u"ɔ̃",u"ɑ̃"]
toipa = dict(zip(ipaIn, ipaOut))

S Z N ê ô â


In [125]:
print coderIPA(u"SâZ piNpôN")

ʃɑ̃ʒ piŋpɔ̃ŋ


In [3]:
rep="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/StemSpaces/LongitudinalesRnd/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/StemSpaces/Longitudinales/"
fichiers=glob.glob(rep+"*X-Regles.pkl")
samples=[f.rsplit("/",1)[-1].split("Regles")[0] for f in fichiers]
samples={int(s.split("-")[1]):s for s in samples}
samples

{1: 'Longitudinal-01-T1000000-F39898-X-',
 2: 'Longitudinal-02-T1000000-F39898-X-'}

In [4]:
sample=samples[1]
fRulesPMS="Regles.pkl"
fRulesPMO="Morphomes-Regles.pkl"

# Classe pour la gestion des patrons, des classes et des transformations

In [5]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print "le numéro de forme n'est pas dans [1,2]",n

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
#                if debug: 
#                    print (forme, file=logfile)
#                    print ("pas de classe",idClasseForme, file=logfile)
#                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
#        else:
#            if debug:
#                print (forme, file=logfile) 
#                print ("pas de patron", file=logfile)
        return sortieForme
        

## Ouvrir les fichiers de règles

In [6]:
def openRules(numero,typeRules="PMS"):
    rules=None
    if typeRules=="PMS" and numero in samples:
        with open(rep+samples[numero]+fRulesPMS, 'rb') as input:
            rules = pickle.load(input)
    elif typeRules=="PMO" and numero in samples:
        with open(rep+samples[numero]+fRulesPMO, 'rb') as input:
            rules = pickle.load(input)
    return rules

In [85]:
numero=1
typeRules="PMO"
avecContexte=True

In [27]:
rules=openRules(numero,typeRules=typeRules)

In [28]:
rules[("ii1S","ii2P")].patrons

{u'E-e': u'^(.*[ptkbdgfsSvzZjrwH][iyEe92a][Jj])E$',
 u'E-ije': u'^(.*[jrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZ][lr])E$',
 u'E-je': u'^(.*[ptkbdgfsSvzZmnJjrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZmnJlr])E$',
 u'HE-je': u'^k\xf4tinyHE$',
 u'wE-je': u'^ZuwE$'}

In [29]:
zeroEntropie=[]
for paire in rules:
    classe=rules[paire].classe
    c1,c2=paire
    if c1!=c2:
        zero=True
        print paire
        for transformation in classe:
            choix=classe[transformation]
            if len(choix)>1:
                print choix
                zero=False
        if zero:
            print "zéro",choix
            zeroEntropie.append(paire)
        print

(u'ps2P', u'is1S')
zéro {u'Hisje-ys': 1}

(u'ii3S', u'ps3P')
{u'E-': 1, u'lE-j': 3}
{u'E-': 1, u'O.E-o.': 42}
{u'E-': 1, u'EjE-wa': 1}
{u'E-': 118, u'jE-': 63}

(u'pc2S', u'ai2P')
zéro {u'6rE-it': 1}

(u'is2S', u'pc3P')
zéro {u'is-rE': 9}

(u'ps2S', u'ii2P')
{u'-je': 120, u'E.-6.je': 91}
{u'-e': 6, u'j-lje': 1}

(u'pc2P', u'ai2P')
zéro {u's6rje-fyt': 1}

(u'ii3P', u'pi2P')
{u'E-e': 1, u'EjE-waje': 1}

(u'fi3P', u'pi2P')
{u'6r\xf4-e': 138, u'E.6r\xf4-6.e': 83}
{u'r\xf4-je': 1, u'r\xf4-ze': 1}
{u'ir\xf4-e': 2, u'r\xf4-se': 4}
{u'ir\xf4-je': 8, u'r\xf4-se': 35}
{u'jEr\xf4-waje': 1, u'jEr\xf4-Eje': 1}
{u'r\xf4-we': 6, u'r\xf4-ve': 1}
{u'ir\xf4-je': 2, u'r\xf4-se': 19}
{u'r\xf4-ze': 14, u'r\xf4-se': 22, u'r\xf4-je': 11}
{u'6r\xf4-Et': 5, u'6r\xf4-e': 34}
{u'r\xf4-je': 7, u'ir\xf4-je': 84, u'r\xf4-se': 8, u'r\xf4-ve': 8}
{u'ir\xf4-e': 12, u'ir\xf4-je': 12, u'r\xf4-se': 44}
{u'ir\xf4-e': 13, u'ir\xf4-je': 3, u'r\xf4-se': 2}

(u'pi2S', u'ps3P')
{u'E-ais': 1, u'E-swa': 1}
{u'-d': 7, u'-v': 3, '

{u'j\xf4-\xf4': 1, u'rj\xf4-j6r\xf4': 1}

(u'ps1P', u'ii1S')
zéro {u'Sj\xf4-vE': 1}

(u'ai1S', u'fi1S')
{u'y-6vrE': 7, u'y-OrE': 2}
{u'i-rE': 3, u'i-ErE': 1, u'-rE': 1}
{u'i-rE': 47, u'-rE': 13}
{u'j.-r.': 24, u'E-6rE': 77}
{u'i-warE': 1, u'i-jErE': 1}
{u'6.E-E.6rE': 103, u'E-6rE': 2}
{u'i-EtrE': 16, u'-rE': 2}
{u'y-ErE': 1, u'y-6vrE': 1}

(u'ii2P', u'ai3S')
{u'je-a': 23, u'je-y': 4}
{u'je-a': 1614, u'je-i': 80}

(u'is2S', u'pI2S')
zéro {u'vis-': 2}

(u'pi3P', u'is3P')
{u'-as': 20, u'-is': 27}

(u'ai3S', u'ii2S')
{u'-zE': 3, u'i-6zE': 5}
{u'-sE': 51, u'i-E': 25}
{u'y-avE': 2, u'y-6vE': 7}
{u'-sE': 5, u'i-wajE': 4, u'i-E': 10}
{u'i-jE': 1, u'-sE': 21, u'i-E': 9}
{u'-sE': 4, u'i-EtE': 8, u'i-E': 3}
{u'-sE': 94, u'i-E': 96}
{u'y-izE': 1, u'y-E': 3}
{u'y-izE': 1, u'y-EzE': 3}
{u'y-EzE': 1, u'y-6vE': 1}
{u'y-wasE': 1, u'y-wajE': 1}

(u'pi2P', u'is2P')
zéro {u'Et-fysje': 1}

(u'is1S', u'pc2P')
zéro {u'\xeas-j\xeadrije': 23}

(u'ai1S', u'is3P')
zéro {u'-s': 66}

(u'pc1S', u'pc2P')
zéro {u'E-i

{u'z-rE': 37, u'-6rE': 54}
{u'-rE': 56, u'-6rE': 956, u'-irE': 27}
{u'-rE': 24, u'-6rE': 1259}
{u'-irE': 4, u'-6rE': 164}
{u'-6rE': 2, u'v-rE': 1}

(u'fi2P', u'ppFP')
{u'E.6re-6.e': 72, u'6re-e': 67}
{u're-t': 6, u'r.-j.': 2}
{u're-t': 8, u'r.-j.': 4, u're-': 45}
{u'r.-j.': 9, u're-': 45}
{u're-t': 4, u'ire-je': 13, u're-': 31}
{u'ire-je': 66, u're-': 44}

(u'is3S', u'pP')
{u'i-\xe2': 10, u'i-waj\xe2': 3, u'-s\xe2': 2}
{u'i-\xe2': 26, u'-s\xe2': 51}
{u'y-Ez\xe2': 2, u'y-iz\xe2': 1}
{u'y-waj\xe2': 1, u'y-was\xe2': 1}
{u'y-\xe2': 2, u'y-iz\xe2': 1}
{u'i-\xe2': 3, u'i-Et\xe2': 8, u'-s\xe2': 4}
{u'a-\xe2': 3, u'ja-\xe2': 19}
{u'i-\xe2': 9, u'-s\xe2': 21, u'i-j\xe2': 1}
{u'y-6v\xe2': 1, u'y-Ez\xe2': 1}
{u'i-6z\xe2': 5, u'-z\xe2': 3}
{u'i-\xe2': 95, u'-s\xe2': 95}

(u'ps2P', u'ppMS')
zéro {u'swaje-Ete': 1}

(u'fi2P', u'is2P')
zéro {u're-jasje': 1}

(u'ii2P', u'ai1P')
{u'je-am': 456, u'je-im': 42}

(u'pi3S', u'pc2S')
{u'-irE': 9, u'-6rE': 39}
{u'-6rE': 4, u'-mirE': 3}
{u'-drE': 51, u'-tirE': 

{u'j.-r.': 17, u'\xf4-6r\xf4': 18}
{u'j.-r.': 46, u'j\xf4-6r\xf4': 1811}
{u'j\xf4-ir\xf4': 14, u'j.-r.': 15, u'j\xf4-6r\xf4': 143}

(u'pI2S', u'ps1P')
zéro {u'9j-ulj\xf4': 1}

(u'ppMP', u'is3S')
{u'y-i': 62, '-': 17}
{'-': 2, u'-zi': 17}

(u'fi2S', u'pc1P')
zéro {u'j6ra-rj\xf4': 1}

(u'pi1P', u'ps2S')
{u'l\xf4-j': 3, u'\xf4-': 1}
{u'j\xf4-': 44, u'\xf4-': 107}
{u'j\xf4-': 1, u'waj\xf4-Ej': 1}
{u'\xf4-': 1, u'Ej\xf4-wa': 1}

(u'ai3P', u'ppMP')
{u'r-': 64, u'ir-y': 52}
{u'r-': 3, u'ir-E': 2}

(u'pi1P', u'ii3P')
zéro {u'waj\xf4-EjE': 1}

(u'pi1P', u'ii1P')
zéro {u'\xf4-j\xf4': 3119}

(u'is3S', u'pc2S')
{u'-rE': 1, u'y-ErE': 5, u'y-irE': 2}
{u'y-6vrE': 7, u'y-OrE': 2}
{u'i-EtrE': 16, u'-rE': 7}
{u'y-watrE': 1, u'y-warE': 1}
{u'-rE': 3, u'i-6rE': 6}
{u'a-6rE': 3, u'6.a-E.6rE': 105}
{u'i-rE': 53, u'-rE': 83}
{u'y-ErE': 1, u'y-6vrE': 1}
{u'-rE': 5, u'i-ErE': 1, u'i-rE': 3}
{u'a-6rE': 132, u'ja-rE': 37}
{u'-rE': 3, u'i-6rE': 2}

(u'ai1S', u'pc2S')
{u'y-6vrE': 7, u'y-OrE': 2}
{u'i-EtrE': 16, u'

zéro {u'-s': 216}

(u'ppFS', u'is2S')
zéro {u'y-is': 24}

(u'ps1P', u'pi2P')
zéro {u'swaj\xf4-Et': 1}

(u'ii3P', u'ii1P')
zéro {u'E-ij\xf4': 47}

(u'is2P', u'fi3S')
{u'j.sje-r.': 1, u'asje-6ra': 1}

(u'ai2P', u'ppFP')
zéro {'-': 1}

(u'pi2P', u'ai2S')
{u'se-': 183, u'e-a': 12}
{u'e-a': 8, u'ave-y': 2}
{u'je-i': 1, u'e-a': 12}
{u'e-a': 123, u'e-y': 6}
{u'ize-y': 2, u'e-a': 79}
{u'e-a': 8, u'waje-i': 5}
{u'e-a': 2782, u'e-i': 134}
{u'e-a': 159, u'e-y': 2}

(u'ai2S', u'inf')
{u'i-Er': 3, u'-r': 3}
{u'y-uvwar': 1, u'y-war': 1}
{u'y-war': 1, u'y-watr': 3}
{u'i-r': 52, u'-r': 65}
{u'i-Etr': 16, u'-r': 7}
{u'y-6vwar': 1, u'y-Er': 1}
{u'i-r': 3, u'i-war': 4, u'-r': 28}
{u'-r': 1, u'y-Er': 5, u'y-war': 3, u'y-ir': 2}
{u'y-avwar': 2, u'y-6vwar': 7}
{u'i-Erir': 1, u'i-r': 2}
{u'i-war': 3, u'-r': 10}

(u'pi1S', u'ii1P')
{u'-dj\xf4': 25, u'-tj\xf4': 4}
{u'-j\xf4': 1, u'-sj\xf4': 16}
{u'-j\xf4': 1, u'-sj\xf4': 35, u'-zj\xf4': 1}
{u'-j\xf4': 7, u'-sj\xf4': 14, u'-vj\xf4': 1}
{u'-j\xf4': 123, u'E.-6.j

{u'ze-r\xf4': 3, u'e-6r\xf4': 78}
{u'6.e-E.6r\xf4': 17, u'e-r\xf4': 6}
{u'e-ir\xf4': 16, u'e-6r\xf4': 173}

(u'fi1P', u'ii3S')
{u'r\xf4-wE': 6, u'r\xf4-vE': 1}
{u'r\xf4-zE': 4, u'ir\xf4-jE': 9, u'r\xf4-sE': 37}
{u'6r\xf4-E': 115, u'E.6r\xf4-6.E': 75}
{u'r\xf4-jE': 7, u'ir\xf4-jE': 11, u'r\xf4-sE': 25, u'r\xf4-vE': 8}
{u'ir\xf4-jE': 2, u'r\xf4-sE': 22}
{u'6r\xf4-E': 26, u'E.6r\xf4-6.E': 8}
{u'r\xf4-E': 15, u'tr\xf4-sE': 13}
{u'ir\xf4-E': 13, u'r\xf4-zE': 1, u'ir\xf4-jE': 12, u'r\xf4-sE': 47}
{u'r\xf4-jE': 9, u'r\xf4-zE': 1}
{u'ir\xf4-E': 9, u'ir\xf4-jE': 1}
{u'ir\xf4-E': 4, u'ir\xf4-jE': 2, u'r\xf4-sE': 7}
{u'r\xf4-zE': 19, u'r\xf4-sE': 24, u'r\xf4-jE': 11}
{u'ir\xf4-E': 2, u'r\xf4-sE': 9}
{u'r\xf4-zE': 5, u'6r\xf4-E': 1890}
{u'r\xf4-zE': 3, u'ir\xf4-jE': 75}

(u'pi1P', u'fi2P')
{u'\xf4-re': 8, u'6.\xf4-E.6re': 16, u'\xf4-6re': 1}
{u'\xf4-re': 30, u'\xf4-ire': 26, u'\xf4-6re': 549}
{u'\xf4-re': 37, u'\xf4-6re': 1647}
{u'z\xf4-re': 12, u'\xf4-6re': 82}
{u'\xf4-ire': 4, u'\xf4-6re': 98}
{

{u'i-EtrE': 16, u'-rE': 7}
{u'y-rE': 50, u'y-irE': 2}

(u'pI2S', u'ai1P')
{u'-tim': 7, u'-am': 136}
{u'wa-ym': 7, u'wa-im': 4}

(u'ii3S', u'ps1S')
{u'E-': 1, u'lE-j': 3}
{u'E-': 1, u'O.E-o.': 42}
{u'E-': 1, u'EjE-wa': 1}
{u'E-': 118, u'jE-': 63}

(u'pc1S', u'fi3S')
{u'j6rE-ra': 10, u'E-a': 16}
{u'E-a': 8, u'rE-j6ra': 7}
{u'E-a': 1, u'jE.E-wa.a': 1}

(u'is1P', u'ii2S')
zéro {u'ysj\xf4-avE': 1}

(u'fi2S', u'ai1S')
{u'6ra-E': 255, u'6ra-i': 2}
{u'ira-jE': 117, u'ra-': 32, u'ra-jE': 4}
{u'ira-jE': 1, u'ira-y': 1}
{u'ira-jE': 5, u'ra-': 4}
{u'E.6ra-6.E': 92, u'6ra-E': 143}

(u'ps3P', u'fi1S')
{u'z-rE': 1, u'-6rE': 3}
{u'-rE': 8, u'-j6rE': 7}
{u'z-rE': 28, u'-6rE': 54}
{u'-rE': 56, u'-6rE': 665, u'-irE': 25}
{u'Ej-jErE': 1, u'Ej-warE': 1}
{u'-rE': 1, u'wa-jErE': 1}
{u'-6rE': 2, u's-trE': 9}
{u'-rE': 22, u'-6rE': 1520}
{u'-irE': 3, u'-6rE': 140}
{u'-6rE': 2, u'v-rE': 1}

(u'ii1P', u'ppMS')
{u'j\xf4-e': 50, u'j\xf4-i': 6}
{u'j\xf4-e': 2, u'sj\xf4-': 29}
{u'j\xf4-e': 35, u'zj\xf4-': 1}
{u'ij\xf

(u'pi3S', u'pc2P')
zéro {u'-rje': 2}

(u'inf', u'is1S')
zéro {u'r-is': 28}

(u'ps1P', u'ps2P')
zéro {u'\xf4-e': 36}

(u'pc3S', u'is2P')
zéro {u'6rE-asje': 1}

(u'ps3S', u'fi3P')
{u'-r\xf4': 8, u'-j6r\xf4': 8}
{u'-r\xf4': 59, u'-6r\xf4': 1779}
{u'z-r\xf4': 26, u'-6r\xf4': 56}
{u'-ir\xf4': 2, u'-6r\xf4': 98}
{u'-ir\xf4': 23, u'-r\xf4': 17, u'-6r\xf4': 413}
{u'-6r\xf4': 21, u's-tr\xf4': 6}

(u'ps1P', u'pi1S')
zéro {u'ulj\xf4-2': 1}

(u'pP', u'is3P')
{u'\xe2-as': 22, u'\xe2-is': 32}

(u'pi3S', u'pI1P')
{u'-\xf4': 129, u'E.-6.\xf4': 96}
{u'-\xf4': 72, u'-d\xf4': 2}
{u'-j\xf4': 8, u'-s\xf4': 5}
{u'-\xf4': 156, u'-t\xf4': 9}
{u'-j\xf4': 1, u'-v\xf4': 4}

(u'pI2P', u'ii3S')
zéro {u'Se-vE': 1}

(u'ps3P', u'pc2P')
{u'-6rje': 106, u'-irje': 5}
{u'z-rje': 1, u'-6rje': 7}
{u'-rije': 34, u'-6rje': 487}
{u'-rije': 1, u'-irje': 1}

(u'ii2P', u'fi1S')
{u'je-irE': 2, u'je-6rE': 48}
{u'zje-rE': 20, u'je-6rE': 83}
{u'je-rE': 1, u'ije-6rE': 8}
{u'je-rE': 27, u'je-irE': 13, u'je-6rE': 150}
{u'6.je-E.6rE': 3

{u'E-': 1, u'O.E-o.': 42}
{u'E-': 1, u'EjE-wa': 1}
{u'E-': 118, u'jE-': 63}

(u'pi1P', u'ii1S')
zéro {u'waj\xf4-EjE': 1}

(u'pi2P', u'pc1P')
zéro {u'6ne-j\xeadri\xf4': 2}

(u'ps3P', u'ppFS')
{u'-e': 320, u'-y': 16, u'-i': 14}
{u'-e': 34, u'E.-6.e': 65}
{u'-e': 59, u'E.-6.e': 6}
{u'-e': 39, u'-i': 6}
{u'-e': 493, u'-y': 41}
{u'-e': 1, u's-': 88}
{u'-je': 12, '-': 2}
{u'-e': 3, u'E.-6.e': 2}
{u'-je': 10, u'wa-y': 4}
{u'-e': 3, u's-': 72}
{u'-e': 2, u'Es-y': 6}

(u'ai2S', u'is3S')
zéro {'-': 4421}

(u'ppFP', u'pc1P')
zéro {u'je-rj\xf4': 1}

(u'is1P', u'ppMS')
zéro {u'sj\xf4-': 1}

(u'fi3S', u'pc1S')
{u'ra-j6rE': 10, u'a-E': 19}
{u'a-E': 9, u'j6ra-rE': 7}

(u'pi3S', u'pi3P')
{u'E-ais': 1, u'E-s\xf4': 1}
{u'-v': 13, '-': 31, u'-s': 32}
{u'-d': 33, '-': 92, u'-t': 4}
{'-': 126, u'-s': 103, u'-z': 11}
{u'-m': 3, '-': 4}
{u'j-': 12, '-': 26}
{u'-t': 2, '-': 49}
{u'-t': 16, '-': 3, u'-s': 41}
{'-': 24, u'-s': 50, u'-z': 6}
{u'je-Ej': 1, u'je-wa': 1}
{u'-d': 7, u'-v': 3, '-': 157, u'-t': 2}
{'-'

In [30]:
zeroPop={}
for paire in zeroEntropie:
    for t in rules[paire].classe:
        pop=0
        for c in rules[paire].classe[t]:
            pop+=rules[paire].classe[t][c]
        zeroPop[paire]=pop
    

In [48]:
stemspaceThresh=3
stemspaceRules=[]
for k in sorted(zeroPop, key=zeroPop.get,reverse=True):
    print k, zeroPop[k]
    if zeroPop[k]>=stemspaceThresh:
        stemspaceRules.append(k)

(u'ppMP', u'ppMS') 4524
(u'ppMS', u'ppMP') 4524
(u'ii3S', u'ii3P') 4508
(u'ii1S', u'ii3S') 4508
(u'ii3P', u'ii2S') 4508
(u'ii3P', u'ii3S') 4508
(u'ii1S', u'ii2S') 4508
(u'ii2S', u'ii3P') 4508
(u'ii1S', u'ii3P') 4508
(u'ii3S', u'ii1S') 4508
(u'ii2S', u'ii1S') 4508
(u'ii3S', u'ii2S') 4508
(u'ii3P', u'ii1S') 4508
(u'ii2S', u'ii3S') 4508
(u'ppFS', u'ppFP') 4433
(u'ppFP', u'ppFS') 4433
(u'is3S', u'ai3S') 4421
(u'ai3S', u'ai2S') 4421
(u'ai3S', u'is3S') 4421
(u'ai2S', u'is3S') 4421
(u'ai2S', u'ai3S') 4421
(u'is3S', u'ai2S') 4421
(u'ppFP', u'ppMS') 4308
(u'ppFP', u'ppMP') 4308
(u'ppFS', u'ppMS') 4308
(u'ppFS', u'ppMP') 4308
(u'ai3P', u'ai1S') 3790
(u'ai1S', u'ai3P') 3790
(u'pP', u'pI1P') 3361
(u'pi1P', u'ii1P') 3119
(u'ii1P', u'ii2P') 2992
(u'ai1P', u'ai3S') 2668
(u'ai2S', u'ai1P') 2668
(u'ai3S', u'ai1P') 2668
(u'ai1P', u'ai2S') 2668
(u'is3S', u'ai1P') 2668
(u'ai1P', u'is3S') 2668
(u'pc2P', u'ii1P') 2594
(u'ai1P', u'ai1S') 2517
(u'ai1S', u'ai1P') 2517
(u'pc1P', u'pc2P') 2000
(u'pc2P', u'pP') 9

(u'pi3P', u'is2P') 1
(u'pI2S', u'is2S') 1
(u'is3P', u'ii2P') 1
(u'is2P', u'ii1P') 1
(u'pc3S', u'pc1P') 1
(u'pc2P', u'ai3P') 1
(u'ps1P', u'pi2S') 1
(u'pc3S', u'is2P') 1
(u'pc3S', u'ps2P') 1
(u'is1S', u'ps1P') 1
(u'ps1P', u'ppFP') 1
(u'is1P', u'fi3S') 1
(u'ai2P', u'ppFP') 1
(u'pc1P', u'ps1P') 1
(u'ps2P', u'ai1P') 1
(u'ps3S', u'pc1P') 1
(u'ps3P', u'ps2P') 1
(u'ps2P', u'pc2S') 1
(u'ps2S', u'pc1P') 1
(u'fi1P', u'ai2P') 1
(u'pc1P', u'is3P') 1
(u'ps2P', u'pc3P') 1
(u'is1P', u'pi2S') 1
(u'is3P', u'pi2P') 1
(u'ai1S', u'pc1P') 1
(u'ii1S', u'ps1P') 1
(u'ppMS', u'is2S') 1
(u'ps1P', u'ps3S') 1
(u'is1S', u'ps2P') 1
(u'pi1P', u'ps2P') 1
(u'is2S', u'fi3S') 1
(u'ppFP', u'ps1P') 1
(u'pI1P', u'pc1P') 1
(u'is2S', u'ps3P') 1


In [49]:
len(stemspaceRules)

297

In [50]:
stemSpace=nx.Graph()
for c1,c2 in stemspaceRules:
    if (c2,c1) in stemspaceRules:
#        print c1,c2
        stemSpace.add_edge(c1,c2)
stemSpaceCliques=list(nx.find_cliques(stemSpace))
len(stemSpaceCliques),sorted(stemSpaceCliques,key=len,reverse=True)

(14,
 [[u'is1S',
   u'is2S',
   u'is3P',
   u'is3S',
   u'ai3S',
   u'ai2S',
   u'ai1P',
   u'ai3P',
   u'ai1S',
   u'is1P'],
  [u'is1S', u'is2S', u'is3P', u'is3S', u'ai3S', u'ai2S', u'ai1P', u'is2P'],
  [u'ai2P', u'ai3S', u'ai1P', u'ai2S', u'ai3P', u'is3S'],
  [u'ii1S', u'ii3S', u'ii2P', u'ii3P', u'ii2S'],
  [u'is1S', u'is2S', u'is3P', u'pc2S'],
  [u'is1S', u'is2S', u'is3P', u'pc3S'],
  [u'is1S', u'is2S', u'is3P', u'pc3P'],
  [u'is1S', u'is2S', u'is3P', u'pc1S'],
  [u'is1S', u'is2S', u'is3P', u'fi1S'],
  [u'ii1P', u'ii2P'],
  [u'ppFS', u'ppFP'],
  [u'pc1P', u'pc2P'],
  [u'ppMP', u'ppMS'],
  [u'ps1P', u'ps2P']])

In [51]:
fSamplePMS="paradigmes.csv"
fSamplePMO="Morphomes-paradigmes.csv"

In [52]:
def openSample(numero,typeSample="PMS"):
    result=None
    fSample=""
    if typeSample=="PMS" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMS
    elif typeSample=="PMO" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMO
    if fSample:
        result=pd.read_csv(rep+samples[numero]+fSamplePMS,sep=";",index_col=0,keep_default_na=False,encoding="utf8")
    return result    

In [79]:
df=openSample(2,typeRules)

In [80]:
def makeZeros(sampleFormes,inFormes=True,contextFree=True):
    series=sampleFormes.copy()
    series={k:v for k,v in series.iteritems() if v!=""}

    paires=rules.keys()
    formesC={}
    for c in series:
        if not c in formesC:
            formesC[c]=set()
        formesC[c].add(series[c])
        if debug: print c, series[c]
        formeC=series[c]
        pairesC=[(c1,c2) for (c1,c2) in paires if c1==c]
        for p in pairesC:
            formeP=rules[p].sortirForme(series[c],contextFree=contextFree)
            if formeP and len(formeP)==1:
                p1=p[1]
                if debug: print p1,formeP
                if p1 not in formesC:
                    formesC[p1]=set()
                formesC[p1].add(formeP.keys()[0])
    if inFormes:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1}
    else:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1 and k not in series}

    return formesZero

In [81]:
def fillStemSpace(sampleFormes,inFormes=True,contextFree=True):
    series=sampleFormes.copy()
    series={k:v for k,v in series.iteritems() if v!=""}

    paires=stemspaceRules
    formesC={}
    for c in series:
        if not c in formesC:
            formesC[c]=set()
        formesC[c].add(series[c])
        if debug: print c, series[c]
        formeC=series[c]
        pairesC=[(c1,c2) for (c1,c2) in paires if c1==c]
        for p in pairesC:
            formeP=rules[p].sortirForme(series[c],contextFree=contextFree)
            if formeP and len(formeP)==1:
                p1=p[1]
                if debug: print p1,formeP
                if p1 not in formesC:
                    formesC[p1]=set()
                formesC[p1].add(formeP.keys()[0])
    if inFormes:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1}
    else:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1 and k not in series}

    return formesZero

In [82]:
cases=[c for c in df.columns.tolist() if c!="lexeme"]
cases=sorted(cases)

In [86]:
lexemes=u"abreuver clore paître distraire être avoir devoir pouvoir pendre perdre prendre".split(" ")
lexemes=[l for l in df["lexeme"].tolist() if not l.endswith(u"er")]
lexemes

[u'abasourdir',
 u'abattre',
 u'abolir',
 u'aboutir',
 u'abrutir',
 u'absoudre',
 u'abstenir',
 u'abstraire',
 u'ab\xe2tardir',
 u'ab\xeatir',
 u'accomplir',
 u'accourcir',
 u'accourir',
 u'accroire',
 u'accroupir',
 u'accro\xeetre',
 u'accueillir',
 u'acqu\xe9rir',
 u'adjoindre',
 u'admettre',
 u'adoucir',
 u'advenir',
 u'affadir',
 u'affaiblir',
 u'affermir',
 u'affranchir',
 u'agir',
 u'agonir',
 u'agrandir',
 u'aguerrir',
 u'ahurir',
 u'aigrir',
 u'alanguir',
 u'alentir',
 u'alourdir',
 u'alunir',
 u'amaigrir',
 u'amerrir',
 u'amincir',
 u'amoindrir',
 u'amollir',
 u'amortir',
 u'anoblir',
 u'an\xe9antir',
 u'apercevoir',
 u'aplanir',
 u'aplatir',
 u'appara\xeetre',
 u'appartenir',
 u'appauvrir',
 u'appendre',
 u'appesantir',
 u'applaudir',
 u'apprendre',
 u'approfondir',
 u'arrondir',
 u'assagir',
 u'assaillir',
 u'assavoir',
 u'asseoir',
 u'asservir',
 u'assombrir',
 u'assortir',
 u'assoupir',
 u'assouplir',
 u'assourdir',
 u'assouvir',
 u'assujettir',
 u'astreindre',
 u'atteindr

In [91]:
dfNew=df.copy()
for lexeme in lexemes:
    index=df.loc[df['lexeme']==lexeme].index.values
    if index:
        i=index[0]
        sampleFormes=df.iloc[i].to_dict()
        sampleFormes={k:v for k,v in sampleFormes.iteritems() if v!=""}
        lexemeI=sampleFormes["lexeme"]
        print lexemeI,
#        formesZero=makeZeros(sampleFormes,inFormes=False)
        formesZero=fillStemSpace(sampleFormes,inFormes=False,contextFree=not avecContexte)
        for case in cases:
            if case in sampleFormes:
#                print "\t",case,"\t  ",sampleFormes[case]
                continue
            elif case in formesZero:
#                print "\t",case,"\t=>",formesZero[case]
                dfNew.iloc[i][case]=formesZero[case]
            else:
#                print "\t",case
                continue
#        print

abasourdir abattre abolir aboutir abrutir absoudre abstenir abstraire abâtardir abêtir accomplir accourcir accourir accroire accroupir accroître accueillir acquérir adjoindre admettre adoucir advenir affadir affaiblir affermir affranchir agir agonir agrandir aguerrir ahurir aigrir alanguir alentir alourdir alunir amaigrir amerrir amincir amoindrir amollir amortir anoblir anéantir apercevoir aplanir aplatir apparaître appartenir appauvrir appendre appesantir applaudir apprendre approfondir arrondir assagir assaillir assavoir asseoir asservir assombrir assortir assoupir assouplir assourdir assouvir assujettir astreindre atteindre attendre attendrir atterrir attiédir avachir avertir avilir avoir bannir barrir battre blanchir bleuir blondir blottir blêmir boire bondir bouffir bouillir brandir bruire brunir bâtir bénir calmir candir ceindre chancir choir choisir chérir circoncire circonscrire circonvenir clapir clore combattre commettre comparaître compatir complaire comprendre compromettre

In [92]:
dfNew

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,,abEsE,,,abEsEr,abEsa,abEs6rô,abEs6rE,abEs6re,...,abEse,abEse,abEse,abEse,,,abEsje,,abEs,abEs
1,abandonner,abâdOnam,abâdOnE,,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,...,abâdOne,abâdOne,abâdOne,abâdOne,abâdOnjô,,,abâdOn,abâdOn,abâdOn
2,abasourdir,abazurdim,abazurdi,abazurdit,abazurdi,abazurdir,abazurdi,abazurdirô,abazurdirE,abazurdire,...,abazurdi,abazurdi,abazurdi,abazurdi,,abazurdis,,abazurdis,,abazurdis
3,abattre,abatim,abati,abatit,abati,abatir,abati,abatrô,abatrE,abatre,...,abaty,abaty,abaty,abaty,,,,abat,,abat
4,abdiquer,abdikam,abdikE,,abdika,abdikEr,abdika,abdik6rô,abdik6rE,abdik6re,...,abdike,abdike,abdike,abdike,abdikjô,,,abdik,abdik,abdik
5,aberrer,,abErE,,abEra,abErEr,abEra,abEr6rô,abEr6rE,abEr6re,...,abEre,abEre,abEre,abEre,abErjô,,,abEr,abEr,abEr
6,abhorrer,abOram,abOrE,,abOra,abOrEr,abOra,abOr6rô,abOr6rE,abOr6re,...,abOre,abOre,abOre,abOre,abOrjô,,,abOr,abOr,abOr
7,abjurer,abZyram,abZyrE,,abZyra,abZyrEr,abZyra,abZyr6rô,abZyr6rE,abZyr6re,...,abZyre,abZyre,abZyre,abZyre,abZyrjô,,,abZyr,abZyr,abZyr
8,abolir,abOlim,abOli,,abOli,abOlir,abOli,abOlirô,abOlirE,abOlire,...,abOli,abOli,abOli,abOli,,abOlis,,abOlis,,abOlis
9,abonder,abôdam,abôdE,,abôda,abôdEr,abôda,abôd6rô,abôd6rE,abôd6re,...,abôde,abôde,abôde,abôde,abôdjô,,,abôd,abôd,abôd


In [42]:
for i in range(500):
    sampleFormes=df.loc[i].to_dict()
    lexemeI=sampleFormes["lexeme"]
    if not lexemeI.endswith("er"):
        print lexemeI
        numero=1
        for k,v in sampleFormes.iteritems():
            if k!="lexeme" and v!="":
                print numero,k,v
                numero+=1
        print
        formesZero=makeZeros(sampleFormes,inFormes=False)
        for k,v in formesZero.iteritems():
            if k!="lexeme":
                print numero,k,v
                numero+=1
        print

abasourdir
1 pP abazurdisâ
2 ai3S abazurdi
3 ai3P abazurdir
4 ps3S abazurdis
5 ii3S abazurdisE
6 inf abazurdir
7 ii3P abazurdisE
8 pi2S abazurdi
9 pi2P abazurdise
10 ppFS abazurdi
11 pI2S abazurdi
12 fi2P abazurdire
13 pi1S abazurdi
14 pi3P abazurdis
15 pi3S abazurdi
16 ppMS abazurdi
17 pc3S abazurdirE
18 pc3P abazurdirE
19 fi3S abazurdira
20 fi3P abazurdirô
21 ppMP abazurdi
22 fi1S abazurdirE
23 pc1S abazurdirE

24 pI1P abazurdisô
25 ii1S abazurdisE
26 ii2S abazurdisE
27 pi1P abazurdisô
28 ps2P abazurdje
29 ppFP abazurdi
30 is2S abazurdis
31 pc1P abazurdirjô
32 is1S abazurdis
33 ai2S abazurdi
34 ps2S abazurdis
35 is3P abazurdis
36 is3S abazurdi
37 ai1P abazurdim
38 ai1S abazurdi
39 pI2P abazurdise
40 ps3P abazurdis
41 pc2P abazurdirje
42 ps1S abazurdis

abattre
1 ii1P abatjô
2 ii1S abatE
3 pP abatâ
4 ai3S abati
5 ai3P abatir
6 ps3S abat
7 ii3S abatE
8 inf abatr
9 ii3P abatE
10 pi2S aba
11 pi2P abate
12 ppFS abaty
13 ppFP abaty
14 fi2P abatre
15 pI2P abate
16 ps2S abat
17 ii2P abatje
1

2 ii1S admEtE
3 pP admEtâ
4 ai3S admi
5 ps3S admEt
6 ii3S admEtE
7 inf admEtr
8 ii3P admEtE
9 pi2S admE
10 ai1S admi
11 pi2P admEte
12 ppFS admiz
13 ppFP admiz
14 pI2S admE
15 fi2P admEtre
16 fi2S admEtra
17 pI2P admEte
18 ps2S admEt
19 ii2P admEtje
20 pi1S admE
21 pi1P admEtô
22 pi3P admEt
23 pi3S admE
24 ppMS admi
25 pc3S admEtrE
26 pc3P admEtrE
27 fi3S admEtra
28 fi3P admEtrô
29 ppMP admi
30 fi1P admEtrô
31 fi1S admEtrE
32 pc1S admEtrE
33 pI1P admEtô

34 ps2P admEsje
35 ai3P admir
36 ai2S admi
37 pc2P admâdrije
38 is3S admi
39 ai1P admim
40 ps3P admEt
41 ii2S admEtE
42 ps1S admEt

adoucir
1 ii1S adusisE
2 pP adusisâ
3 ai3S adusi
4 ai3P adusir
5 ps3S adusis
6 ii3S adusisE
7 inf adusir
8 ii3P adusisE
9 pi2S adusi
10 pi2P adusise
11 ppFS adusi
12 ppFP adusi
13 pI2S adusi
14 fi2P adusire
15 fi2S adusira
16 pi1S adusi
17 pi3P adusis
18 pi3S adusi
19 ppMS adusi
20 pc3S adusirE
21 pc3P adusirE
22 fi3S adusira
23 fi3P adusirô
24 ppMP adusi
25 fi1P adusirô
26 fi1S adusirE
27 pc1S adusirE

28

42 ps1S amEgris

amerrir
1 pP amErjâ
2 ai3S amErja
3 ai3P amErjEr
4 ii3S amErjE
5 inf amErje
6 ii3P amErjE
7 pi2S amEri
8 ai1S amErjE
9 pi2P amErje
10 ppFS amErje
11 ppFP amErje
12 pI2S amEri
13 fi2P amErire
14 fi2S amErira
15 pi1S amEri
16 pi1P amErjô
17 pi3P amEri
18 pi3S amEri
19 ppMS amErje
20 pc3S amErirE
21 fi3S amErira
22 ppMP amErje
23 fi1P amErirô
24 fi1S amErirE
25 pc1S amErirE

26 ii1S amErjE
27 ii2S amErjE
28 ps2S amEri
29 is2S amErjas
30 is1S amErjas
31 pc1P amErrjô
32 ai2S amErja
33 is3P amErjas
34 is3S amErja
35 ai1P amErjam
36 ps3S amEri
37 ps3P amEri
38 pI2P amErje
39 pI1P amErjô
40 ps1S amEri

amincir
1 ii1S amêsisE
2 pP amêsisâ
3 ai3S amêsi
4 ai3P amêsir
5 ps3S amêsis
6 ii3S amêsisE
7 inf amêsir
8 ii3P amêsisE
9 pi2S amêsi
10 pi2P amêsise
11 ppFS amêsi
12 ppFP amêsi
13 pI2S amêsi
14 fi2P amêsire
15 pi1S amêsi
16 pi3P amêsis
17 pi3S amêsi
18 ppMS amêsi
19 pc3S amêsirE
20 pc3P amêsirE
21 fi3S amêsira
22 fi3P amêsirô
23 ppMP amêsi
24 fi1S amêsirE
25 pc1S amêsirE

26 pI1

26 pi1P aprOfôdisô
27 ps2P aprOfôdje
28 ps2S aprOfôdis
29 is2S aprOfôdis
30 pc1P aprOfôdirjô
31 ai3S aprOfôdi
32 is1S aprOfôdis
33 ai2S aprOfôdi
34 is3P aprOfôdis
35 is3S aprOfôdi
36 ai1P aprOfôdim
37 ai1S aprOfôdi
38 pI2P aprOfôdise
39 ps3P aprOfôdis
40 pc2P aprOfôdirje
41 ps1S aprOfôdis

arrondir
1 pP arôdisâ
2 ai3S arôdi
3 ai3P arôdir
4 ps3S arôdis
5 ii3S arôdisE
6 inf arôdir
7 ii3P arôdisE
8 pi2S arôdi
9 ppFS arôdi
10 ppFP arôdi
11 fi2P arôdire
12 fi2S arôdira
13 pi1S arôdi
14 pi3P arôdis
15 pi3S arôdi
16 ppMS arôdi
17 pc3S arôdirE
18 fi3S arôdira
19 fi3P arôdirô
20 ppMP arôdi
21 fi1P arôdirô
22 fi1S arôdirE

23 pI1P arôdisô
24 ii1S arôdisE
25 ii2S arôdisE
26 pi1P arôdisô
27 ps2P arôdje
28 ps2S arôdis
29 is2S arôdis
30 pc1P arôdirjô
31 is1S arôdis
32 ai2S arôdi
33 is3P arôdis
34 is3S arôdi
35 ai1P arôdim
36 ai1S arôdi
37 pI2P arôdise
38 pI2S arôdi
39 ps3P arôdis
40 pc2P arôdirje
41 ps1S arôdis

assagir
1 ii1P asaZisjô
2 ii1S asaZisE
3 pP asaZisâ
4 ai3S asaZi
5 ai3P asaZir
6 is3S as

13 pi3S atâdri
14 ppMS atâdri
15 pc3S atâdrirE
16 pc3P atâdrirE
17 ppMP atâdri

18 pI1P atâdrisô
19 ii2S atâdrisE
20 pi1P atâdrisô
21 ps2P atâdrje
22 ppFP atâdri
23 is2S atâdris
24 is1S atâdris
25 pc1P atâdrirjô
26 ai2S atâdri
27 ps2S atâdris
28 is3P atâdris
29 is3S atâdri
30 ps3S atâdris
31 ai1P atâdrim
32 ps3P atâdris
33 pI2P atâdrise
34 pI2S atâdri
35 ai1S atâdri
36 pc2P atâdrirje
37 ps1S atâdris

atterrir
1 pP atErisâ
2 ai3S atEri
3 ai3P atErir
4 ps3S atEris
5 ai1P atErim
6 inf atErir
7 ii3P atErisE
8 pi2S atEri
9 fi2P atErire
10 fi2S atErira
11 pi1S atEri
12 pi3P atEris
13 pi3S atEri
14 ppMS atEri
15 pc3S atErirE
16 fi3S atErira
17 fi3P atErirô
18 ppMP atEri
19 fi1P atErirô
20 fi1S atErirE
21 pc1S atErirE

22 pI1P atErisô
23 ii1S atErisE
24 ii2S atErisE
25 pi1P atErisô
26 ps2S atEris
27 is2S atEris
28 is1S atEris
29 ai2S atEri
30 is3P atEris
31 is3S atEri
32 ii3S atErisE
33 ai1S atEri
34 pI2P atErise
35 pI2S atEri
36 ps3P atEris
37 ps1S atEris

attiédir
1 ii1P atjEdisjô
2 ii1S atj

In [ ]:
rules[("pi3S","ii1S")].sortirForme("Xi",contextFree=True)